In [0]:
%run ./Invoice_json_streaming_module

In [0]:
class TestSuite:

    def __init__(self):
        self.invoice_dir = '/FileStore/invoices'
    
    def cleaning_dir(self):
        print('Cleaning Directory...', end='')
        spark.sql('drop table if exists invoice_table')
        dbutils.fs.rm(f'{self.invoice_dir}/incoming', recurse=True)
        dbutils.fs.rm(f'{self.invoice_dir}/checkpoint', recurse=True)
        dbutils.fs.rm('/user/hive/warehouse/invoice_table', recurse=True)
        dbutils.fs.mkdirs(f'{self.invoice_dir}/incoming')
        print('Done')
    
    def copying_files(self, file_name):
        dbutils.fs.cp(f'{self.invoice_dir}/landing/{file_name}', f'{self.invoice_dir}/incoming', recurse=True)
    
    def test_case(self):
        self.cleaning_dir()
        invoice_obj = Invoice()

        print('Testcase 1:')
        self.copying_files(file_name = 'invoices_1.json')
        invoice_obj.launcher()
        val = spark.sql('select count(*) from invoice_table').collect()[0][0]
        assert val==1253, f'Testcase 1 failed, returned value is {val}'
        print('Passed.\n')

        print('Testcase 2:')
        self.copying_files(file_name = 'invoices_2.json')
        invoice_obj.launcher()
        val = spark.sql('select count(*) from invoice_table').collect()[0][0]
        assert val==3763, f'Testcase 2 failed, returned value is {val}'
        print('Passed.\n')

        print('Testcase 3:')
        self.copying_files(file_name = 'invoices_4.json')
        invoice_obj.launcher()
        val = spark.sql('select count(*) from invoice_table').collect()[0][0]
        assert val==6295, f'Testcase 3 failed, returned value is {val}'
        print('Passed.')

In [0]:
obj = TestSuite()
obj.test_case()

Cleaning Directory...Done
Testcase 1:
Invoice Batch load started...Done.
Passed.

Testcase 2:
Invoice Batch load started...Done.
Passed.

Testcase 3:
Invoice Batch load started...Done.
Passed.
